In [1]:
# import the necessary libraries

import numpy as np
import pandas as pd
import os
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings("ignore")

In [6]:
# unzipping the file if required

path = os.getcwd()
path = os.path.join(path, "all_data.zip")
!unzip {path} -d "/content/"

Archive:  /content/all_data.zip
  inflating: /content/all_data/test_c1000_d100.csv  
  inflating: /content/all_data/test_c1000_d1000.csv  
  inflating: /content/all_data/test_c1000_d5000.csv  
  inflating: /content/all_data/test_c1500_d100.csv  
  inflating: /content/all_data/test_c1500_d1000.csv  
  inflating: /content/all_data/test_c1500_d5000.csv  
  inflating: /content/all_data/test_c1800_d100.csv  
  inflating: /content/all_data/test_c1800_d1000.csv  
  inflating: /content/all_data/test_c1800_d5000.csv  
  inflating: /content/all_data/test_c300_d100.csv  
  inflating: /content/all_data/test_c300_d1000.csv  
  inflating: /content/all_data/test_c300_d5000.csv  
  inflating: /content/all_data/test_c500_d100.csv  
  inflating: /content/all_data/test_c500_d1000.csv  
  inflating: /content/all_data/test_c500_d5000.csv  
  inflating: /content/all_data/train_c1000_d100.csv  
  inflating: /content/all_data/train_c1000_d1000.csv  
  inflating: /content/all_data/train_c1000_d5000.csv  
  inf

In [7]:
# creating a dataframe to store results
df_results = pd.DataFrame(columns=['DataSet_name', 'N_Estimators', 'Criterion', 'Max_depth', 'Min_samples_split', 'Accuracy', 'F1_score'])

In [8]:
# Defining the Random Forest Classifier

def DT():

  # Getting the features and output 
  XValid = validData.iloc[:, 0:-1].values
  YValid = validData.iloc[:, -1].values
  XTest = testData.iloc[:, 0:-1].values
  YTest = testData.iloc[:, -1].values

  # Getting the best parameters
  parameters = {'n_estimators': (50, 100, 500), 'criterion': ('gini', 'entropy'), 'max_depth': (None, 10, 50, 100), 'min_samples_split': (2, 10, 15)}
  grid = GridSearchCV(RandomForestClassifier(), parameters, n_jobs=10, verbose=True)
  grid.fit(XValid, YValid)

  # Combining training and validation data
  trainX1 = trainData.iloc[:, 0:-1]
  validX1 = validData.iloc[:, 0:-1]
  XData = pd.concat([trainX1, validX1], axis=0, copy=True).values

  trainY1 = trainData.iloc[:, -1]
  validY2 = validData.iloc[:, -1]
  YData = pd.concat([trainY1, validY2], axis=0, copy=True).values

  # Final training and Testing
  tree = RandomForestClassifier(**grid.best_params_)
  tree = tree.fit(XData, YData)
  yPredict = tree.predict(XTest)

  return grid.best_params_, metrics.accuracy_score(YTest, yPredict), f1_score(YTest, yPredict) 

In [ ]:
# Getting the data
path = os.getcwd()
path = os.path.join(path, 'all_data')
firstNum = [3, 5, 10, 15, 18]
secNum = [1, 10, 50]

for i in firstNum:
  for j in secNum:
    trainData = pd.read_csv(path + "/train_c" + str(i) + "00_d" + str(j) + "00.csv", header=None)
    validData = pd.read_csv(path + "/valid_c" + str(i) + "00_d" + str(j) + "00.csv", header=None)
    testData = pd.read_csv(path + "/test_c" + str(i) + "00_d" + str(j) + "00.csv", header=None)

    Best_Parameters, Acc, F_Score = DT()
    df_results = df_results.append({'DataSet_name': "c" + str(i) + "00_d" + str(j) + "00", 'N_Estimators': Best_Parameters['n_estimators'], 'Criterion': Best_Parameters['criterion'], 'Max_depth': Best_Parameters['max_depth'], 'Min_samples_split': Best_Parameters['min_samples_split'], 'Accuracy': Acc, 'F1_score': F_Score}, ignore_index=True)

In [ ]:
df_results

,DataSet_name,N_Estimators,Criterion,Max_depth,Min_samples_split,Accuracy,F1_score
0,c300_d100,500,gini,100,10,0.8500,0.852941
1,c300_d1000,500,entropy,10,10,0.9075,0.908006
2,c300_d5000,500,entropy,50,10,0.9366,0.937697
3,c500_d100,500,entropy,None,15,0.9250,0.923077
4,c500_d1000,500,entropy,10,2,0.9580,0.958209
5,c500_d5000,500,entropy,10,10,0.9566,0.957325
6,c1000_d100,500,gini,None,2,0.9950,0.994975
7,c1000_d1000,500,gini,50,10,0.9925,0.992489
8,c1000_d5000,500,gini,100,2,0.9964,0.996403
9,c1500_d100,50,gini,None,2,1.0000,1.000000
